### If you are reading this notebook on the GitHub, please go to [README](./README.md) and follow installation instructions to set everything up locally, it's an interactive notebook and you need a local setup to execute the cells. 

---

# CS 6601: Artificial Intelligence - Assignment 1 - Search

Search is an integral part of AI. It helps in problem solving across a wide variety of domains where a solution isn’t immediately clear.  You will implement several graph search algorithms with the goal of solving bi-directional and tri-directional search.

# Table of Contents

0. [About the Assignment](#about-the-assignment)
1. [Submission Instructions](#submission-instructions)
2. [Important Files](#important-files)
3. [Grading](#grading)
4. [Frequently Asked Questions](#faq)
5. [Resources](#resources)
6. [Coding Time!](#coding-time)
7. [Race!](#race)

# About the Assignment <a name="about-the-assignment"></a>

Your task is to implement several search algorithms that will calculate a route between two points in Romania while seeking to minimize time and space cost. We will be using an undirected network representing a map of Romania (and an optional Atlanta graph used for the Race!).

![romania.png](romania/romania.png)

# Submission Instructions <a name="submission-instructions"></a>

The grade you receive for the assignment will be determined as follows:

| Section | Points    | Condition |
| ------- | --------- | --------- |
| [Name](#name) | 1 point | Return your name. |
| [Warmup 1](#priority-queue) | 5 points | Implement a Priority Queue from scratch or by modifying Python's `heapq` library. |
| [Warmup 2](#bfs) | 5 points | Implement Breadth First Search (BFS). |
| [Warmup 3](#ucs) | 10 points | Implement Uniform Cost Search (UCS). |
| [Warmup 4](#astar) | 10 points | Implement A* Search. |
| [Exercise 1](#bi-ucs) | 20 points | Implement Bidirectional UCS Search. |
| [Exercise 2](#bi-astar) | 29 points | Implement Bidirectional A* Search. |
| [Exercise 3](#tri-ucs) | 12 points | Implement Tridirectional UCS Search. |
| [Exercise 4](#tri-upgraded) | 8 points | Implement Upgraded Tridirectional Search. |
| [Race!](#race) | 5 points (Extra Credit) | Implement your best search algorithm for a competition. |

All code you will edit is in the `notebook.ipynb` file. In order to obtain your submission file for Gradescope, **save your notebook, then uncomment and run the code cell below.**

In [466]:
%run helpers/notebook2script submission

Late Policy:
    
      "I have read the late policy for CS3600/6601."
    
Please type 'yes' to agree and continue>yes


Honor Pledge:
    
      "I have read the Collaboration and Academic Honesty policy for CS3600/6601.
      I certify that I have or will use outside references only in accordance with
      this policy, that I have or will cite any such references via code comments,
      and that I have not or will not copy any portion of my submission from another
      past or current student."

    
Please type 'yes' to agree and continue>yes


Converted notebook.ipynb to submission/submission.py


The above code cell will generate a `submission/submission.py` file. Do **NOT** modify this file, directly submit it to Gradescope for grading. You are allowed **two submissions every thirty minutes**. In your Gradescope submission history, you can mark a certain submission as 'Active'. Gradescope automatically marks your most recent submission as the one for grading, so make sure you activate your best submission for grading at the end. The autograder is set to timeout in **10 minutes**, so if your submissions runs for longer than 10 minutes it will be terminated and not graded. Note that in some cases, this will fail to occur due to infinite loops or memory overflow, in which case you will have crased the autograder and should reach on Edstem to see if the teaching staff can manually terminate the autograder for you.

**Do NOT erase the `#export` at the top of any cells as it is used by `notebook2script.py` to extract cells for submission.**

# Important Files <a name="important-files"></a>

While you'll only have to edit `notebook.ipynb` and submit the exported `submission.py`, there are a number of other notable files:

1. `notebook.ipynb`: Where you'll implement the required methods for your search algorithms.
2. `search_basic_tests.py`: Sample basic tests, visualizes the Romania graph.
3. `search_submission_tests_grid.py`: Visualizes the search as a grid.
4. `search_romania_tests.py`: More comprehensive tests on the Romania graph than `search_basic_tests`.
5. `search_atlanta_tests.py`: Tests for the Atlanta graph.
6. `search_case_visualizer.py`: For visualizing specific failed cases of interest on the Romania graph.

Feel free to play around with provided tests above and to write your own for more comprehensive testing. See the README file for details on how to run them.

#### Visualizing the Atlanta graph:

The Atlanta graph is used in the extra credit part of this assignment. However, it is too big to display within a Python window like Romania. As a result, when you run the bidirectional tests in **_search_atlanta_tests.py_**, it generates a JSON file in the GeoJSON format. To see the graph, you can upload it to a private GitHub Gist or use [this](http://geojson.io/) site.

If you want to see how **_visualize_graph.py_** is used, take a look at the test functions like `test_bi_ucs_atlanta_custom` in **_search_atlanta_tests.py_**

# Grading <a name="grading"></a>

Points for each section are awarded based on finding the most optimal path (for that search algorithm) and then evaluating the number of nodes explored. To track the number of times a node is explored during the search, the `ExplorableGraph` wrapper is used on the networkx Graph class. Every time you process a node, by calling `graph.neighbors(node)`, the count for that node increases by one. You will need to use one of these methods to add a node's neighbors to the search queue, just be careful not to call it unnecessarily throughout your code. We have created the `graph.get_edge_weight(u, v)` method to be used to access edge weights between two nodes, `u` and `v`. All other normal `networkx` Graph operations can be performed. Exercise specific details can be found in the exercise descriptions below.

# Frequently Asked Questions <a name="faq"></a>

> **General**
> 1. Do **NOT** create a copy of the graph structure for any of the algorithms or computations. We refer to this as "caching neighbors" and it is illegal.
> 2. A walkthrough video will be released with the assignment release on Edstem. It is highly recommended that you watch the video for guidance on how to get started on the project as well as tips for individual exercises.
> 3. This is a difficult assignment! Get started early, use office hours, and don't give up!

> **Error Messages (Gradescope and Local)**
> 1. **NoneType object is not subscriptable** - This usually occurs when you return a path that is not completely connected (a pair of successive nodes in your path are not connected) or if your path returned is empty when it is not supposed to be empty.
> 2. **Path does not go from start to goal or is invalid** - (Gradescope) The path returned by your algorithm does not have the start and end goals at their respective positions; does not have the correct start and end goals; is not the shortest path to the goal; or is not a complete/connected path.
> 3. **Path is longer than optimal path** - (Gradescope) Your path cost is greater than the expected optimal path cost/benchmark.
> 4. **Nodes explored should be from a valid frontier and should be kept to a minimum** - (Gradescope) Your algorithm is exploring more nodes than necessary or is exploring nodes that should not be explored in that particular test. This is independent of whether or not your path is correct.
> 5. **On average for the failed test case(s), excluding cases where path is longer than optimal: 
number of nodes explored more than the benchmark = xyz** - (Gradescope) The average number of nodes explored in excess by your algorithm (includes only cases where path was optimal)
> 6. **Path cost is incorrect** - (Gradescope) The cost of your path is incorrect.
> 7. **List index was out of range** - (Gradescope) This is likely due to the fact that your code may not be generalized properly. While we provide the Romania map for local testing, the map on Gradescope is not identical and node labels may not be one character strings. (i.e. multicharacter strings, integers, floats, etc.) Make sure your code is generalized!

> **Grading**
> 1. **The grade you see in Gradecope is the grade that you will get, unless an OSI violation is detected**. In the event that an OSI violation is detected, you will be notified at some point after the assignment closure for administrative steps. So please, check with teaching staff before you do anything that might result in an OSI violation.
> 2. **Are in-person and OMSCS folk graded together?** No. They are two separate sections.

# Resources <a name="resources"></a>

General resources:
* Canvas Course Videos: Search Module
* [R&N slides on Uninformed Search](https://www.cc.gatech.edu/~thad/6601-gradAI-fall2015/chapter03-clean.pdf)
* [Informed Search](https://www.cc.gatech.edu/~thad/6601-gradAI-fall2015/chapter04a.pdf)
* [Comparing BFS and DFS](https://cs.stanford.edu/people/abisee/tutorial/bfsdfs.html)
* [A* Search](https://cs.stanford.edu/people/abisee/tutorial/astar.html)

Advanced resources:
* [A Star meets Graph Theory](https://github.gatech.edu/omscs6601/assignment_1/raw/master/resources/A%20Star%20meets%20Graph%20Theory.pdf)
* [Bi Directional A Star - Slides](https://github.gatech.edu/omscs6601/assignment_1/raw/master/resources/Bi%20Directional%20A%20Star%20-%20Slides.pdf)
* [Bi Directional A Star with Additive Approx Bounds](https://github.gatech.edu/omscs6601/assignment_1/raw/master/resources/Bi%20Directional%20A%20Star%20with%20Additive%20Approx%20Bounds.pdf)
* [Bi Directional A Star](https://github.gatech.edu/omscs6601/assignment_1/raw/master/resources/Bi%20Directional%20A%20Star.pdf)
* [Search Algorithms Slide Deck](https://github.gatech.edu/omscs6601/assignment_1/raw/master/resources/Search%20Algorithms%20Slide%20Deck.pdf)
* [Bi Directional Stopping Conditions, Piazza '17](https://docs.google.com/document/d/14Wr2SeRKDXFGdD-qNrBpXjW8INCGIfiAoJ0UkZaLWto/pub)
* [Bi Directional Search Visualizations](https://drive.google.com/file/d/1SxhOnAn4uAI17HdTq082PuzQ_jZnp4Nw/view?usp=sharing)
* [Piazza: Landmark Example](https://docs.google.com/document/d/1YEptGbSYUtu180MfvmrmA4B6X9ImdI4oOmLaaMRHiCA/pub)

Interesting reads - links from Canvas, below the videos:
* [Finding Optimal Solutions to Rubik's Cube Using Pattern Databases](https://www.cs.princeton.edu/courses/archive/fall06/cos402/papers/korfrubik.pdf)
* [God's Number is 26 in the Quarter-Turn Metric](http://www.cube20.org/qtm/)
* [Reach for A∗: An Efficient Point-to-Point Shortest Path Algorithm](http://www.cc.gatech.edu/~thad/6601-gradAI-fall2015/02-search-01-Astart-ALT-Reach.pdf)
* [Computing the Shortest Path: A∗ Search Meets Graph Theory](http://www.cc.gatech.edu/~thad/6601-gradAI-fall2015/02-search-Goldberg03tr.pdf)
* [Reach-based Routing: A New Approach to Shortest Path Algorithms Optimized for Road Networks](http://www.cc.gatech.edu/~thad/6601-gradAI-fall2015/02-search-Gutman04siam.pdf)


**_Please refrain from referring to code/psuedocode from other resources aside from these._**

# Coding Time! <a name="coding-time"></a>

## Verify Setup

In [ ]:
# %run helpers/verify_config.py # verify the environment setup

## Importing External Modules

In [1]:
# Following two lines make sure anything imported from .py scripts͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁 
# is automatically reloaded if edited & saved (e.g. local unit tests or players)͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
%load_ext autoreload
%autoreload 2

**Do NOT modify the cell below! You are only allowed to use the imports in the cell below for this assignment.**

In [2]:
#export
import heapq
import os
import pickle
import math

If you have discussed this assignment at a whiteboard level, got help from Edstem or have used external resources (not provided by the instructors) that you may want to cite, please do so in the cell below as a Python comment! (no need to cite Python or included packages documentation) Make sure that you have checked with the teaching staff whether or not a resources is allowed **BEFORE** you reference that resource!

<font color='darkred'>
* Plagiarism is a **serious offense**. You are responsible for completing your own work. You are not allowed to copy and paste, or paraphrase, or submit materials created or published by others, as if you created the materials. All materials submitted must be your own.
</font>

<font color='darkred'>
* All incidents of suspected dishonesty, plagiarism, or violations of the Georgia Tech Honor Code will be subject to the institute’s Academic Integrity procedures. If we observe any (even small) similarities/plagiarisms detected by Gradescope or our TAs, **WE WILL DIRECTLY REPORT ALL CASES TO OSI**, which may, unfortunately, lead to a very harsh outcome. **Consequences can be severe, e.g., academic probation or dismissal, grade penalties, a 0 grade for assignments concerned, and prohibition from withdrawing from the class.**
</font>

In [ ]:
#export
# Credits if any͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
# 1)͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁 Warmup 1: I used the idea on how to preserve FIFO from https://docs.python.org/3/library/heapq.html.
# 2)͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
# 3)͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁

## Your Name (1 Pt) <a name="important-files"></a>

A simple task to begin the assignment. Return your name from the function aptly called `return_your_name()`.

In [ ]:
#export

def return_your_name() -> str:
    """Return your first and last name from this function as a string"""
    # TODO: finish this function͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    return "Franz Adam"
    raise NotImplementedError
    

In [ ]:
# Local test͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
print(return_your_name())

## Warmups

We'll start by implementing some simpler optimization and search algorithms before the real exercises.


### Warmup 1: Priority Queue (5 Pts) <a name="priority-queue"></a>

In all searches that involve calculating path cost or heuristic (e.g. uniform-cost), we have to order our search frontier. It turns out the way that we do this can impact our overall search runtime.

To show this, you'll implement a priority queue which will help you in understanding its performance benefits. For large graphs, sorting all input to a priority queue is impractical. As such, the data structure you implement should have an **amortized O(1) insertion and O(lg n) removal time**. It should do better than the naive implementation in our tests (InsertionSortQueue), which sorts the entire list after every insertion. Note that for this assignment, we treat **smaller values as values with higher priority**. For example a value of 1 has a higher priority than a value of 2.

> **Requirements**
> 1. Amortized O(1) insertion and O(log n) removal. What common data structure has these characteristics?
> 2. Implement the functions `pop` and `append` in the `PriorityQueue` class below at a minimum. You can add extra helper functions with the class if you'd like and you may also implement `remove` if you find that necessary.
> 3. Preserve FIFO. It is possible for the priority queue to receive two elements with the same priority. In the event that that occurs, the element that joined the priority queue first should be returned first.
> 4. Generalize. The nodes provided to the priority queue will be Python tuples in the form of `(priority, payload)`. You may assume that the datatype for `priority` is a standard Python datatype (i.e. `int`, `float`, `str`, etc.). The `payload` can be of any datatype (standard or custom).
> 5. It is possible for duplicate nodes to enter the queue. (i.e. identical priority, identical payload)

> **FAQ**
> 1. **What will be input to the Priority Queue?** We feed nodes in the form `(priority, value)` and expect to receive the nodes back in that same form.
> 1. **How do I preserve FIFO?** The best hint we can give you is to think about using some sort of a counter to keep track of when each node joined the Priority Queue...
> 3. **Can I use Heapq?** Yes. In fact those of you more familiar with Python may find it much easier to just modify `heapq` to satisfy our requirements than to implement a Priority Queue from scratch (as `heapq` does not satisfy the FIFO requirement). However you have to reference `heapq` documentation yourself. The value in this exercise comes from being able to implement a Priority Queue from scratch. (You might see something like this in a coding interview!)
> 4. **Are the local tests comprehensive?** No. Passing local tests does not guarantee that you will pass Gradescope. Try thinking about different scenarios and writing your own tests too!

In [73]:
#export

class PriorityQueue(object):
    """
    A queue structure where each element is served in order of priority.

    Elements in the queue are popped based on the priority with higher priority
    elements being served before lower priority elements.  If two elements have
    the same priority, they will be served in the order they were added to the
    queue.

    Traditionally priority queues are implemented with heaps, but there are any
    number of implementation options.

    (Hint: take a look at the module heapq)

    Attributes:
        queue (list): Nodes added to the priority queue.
    """

    def __init__(self):
        """Initialize a new Priority Queue."""

        self.queue = []
        self.counter = 0

    def pop(self):
       
        """
        Pop the top priority node from the queue.

        Returns:
        The node with the highest priority.
        """
        if self.size() == 0: #Check for empty
            raise IndexError("There are no items in the queue.")
        
        elif self.size() == 1: #I hardcoded this base case to simplify the implementation
            return_node = self.queue[0]
            self.queue.pop(0)
        
        else: 
            return_node = self.queue[0]
            self.queue[0], heapify_index = self.queue[-1], 0 #Swap last and first item and then heapify it downwards
            self.queue.pop(-1) #Delete/Pop the last item 
        
            index_left_child = heapify_index * 2 + 1 #Set up child indixes for heapified node
            index_right_child = heapify_index * 2 + 2
        
            #While left or right children exist to potentially swap
            while index_left_child < self.size() or index_right_child < self.size(): 
                smallest = heapify_index

                #If left child exists and < than heapified node or if == same and counter is smaller, set smallest to left child
                if index_left_child < self.size() and ((self.queue[index_left_child][0] < self.queue[smallest][0]) 
                                                       or ((self.queue[index_left_child][0] == self.queue[smallest][0]) 
                                                           and (self.queue[index_left_child][1] < self.queue[smallest][1]))):
                    smallest = index_left_child
                #Same for the right
                if index_right_child < self.size() and ((self.queue[index_right_child][0] < self.queue[smallest][0]) 
                                                       or ((self.queue[index_right_child][0] == self.queue[smallest][0]) 
                                                           and (self.queue[index_right_child][1] < self.queue[smallest][1]))):
                    smallest = index_right_child

                if smallest == heapify_index: #Break if heapified node 
                    break

                # Swap with the smallest child
                self.queue[heapify_index], self.queue[smallest] = self.queue[smallest], self.queue[heapify_index]
                heapify_index = smallest

                index_left_child = heapify_index * 2 + 1
                index_right_child = heapify_index * 2 + 2

        return (return_node[0], return_node[2])

    def remove(self, node):
        """
        Remove a node from the queue.

        Hint: You might require this in ucs. However, you may
        choose not to use it or to define your own method.

        Args:
            node (tuple): The node to remove from the queue.
        """

        raise NotImplementedError
        
    def update(self, node, updated_priority):
        """
        Updates priority value of queue for A* and UCS to a smaller value if found in A*
        Loops through and updates. 
        """
        loop = True
        
        for i in range(len(self.queue)):
            
            if (self.queue[i][2] == node) and (loop == True):
                #current = node
                self.queue[i][0] = updated_priority
                
                if i != 0:
                    current_index = i
                    parent_index = int((current_index - 1) // 2)
                
                    #Heapfiy up
                    while (((self.queue[current_index][0] < 
                           self.queue[parent_index][0]) or (self.queue[current_index][0] == 
                           self.queue[parent_index][0] and self.queue[current_index][1] < 
                           self.queue[parent_index][1])) and (current_index != 0)):
                    
                        temp = self.queue[current_index]
                        self.queue[current_index] = self.queue[parent_index]
                        self.queue[parent_index] = temp
                           
                        #update current and parent index
                        current_index = parent_index
                        if current_index != 0:
                           parent_index = int((parent_index - 1) // 2)
                           
                    loop = False

    def __iter__(self):
        """Queue iterator."""

        return iter(sorted(self.queue))

    def __str__(self):
        """Priority Queue to string."""

        return 'PQ:%s' % self.queue

    def append(self, node):
        """
        Append a node to the queue.

        Args:
            node: Comparable Object to be added to the priority queue.
        
        I am building a min-heap as implementation. I conserve the min-heap property by heapifying every new node 
        as long as it's smaller than it's parent.
        """
        #Restructure the tuple into a list to keep track of insertion order
        node_list = []
        node_list.append(node[0])
        node_list.append(self.counter)
        node_list.append(node[1])
        self.counter = self.counter + 1
        
        if self.size() == 0:
            self.queue.append(node_list)
        else: 
            self.queue.append(node_list)
            new_node_index = self.size() - 1
            parent_index = int((new_node_index - 1) // 2)
            
            while((parent_index >= 0 and (node_list[0] < self.queue[parent_index][0])) 
                  or (node_list[0] == self.queue[parent_index][0] 
                      and node_list[1] < self.queue[parent_index][1])): #Heapify, while parent is bigger than inserted node, swap them
                
                temp_parent_node = self.queue[parent_index]
                self.queue[parent_index] = node_list
                self.queue[new_node_index] = temp_parent_node
                new_node_index = parent_index
                parent_index = int((parent_index -1) // 2) #Will reach -1 if new node has lowest priority, adding and conditional
            
        return "Append Successful"

    def __contains__(self, key):
        """
        Containment Check operator for 'in'

        Args:
            key: The key to check for in the queue.

        Returns:
            True if key is found in queue, False otherwise.
        """

        return key in [n[-1] for n in self.queue]

    def __eq__(self, other):
        """
        Compare this Priority Queue with another Priority Queue.

        Args:
            other (PriorityQueue): Priority Queue to compare against.

        Returns:
            True if the two priority queues are equivalent.
        """

        return self.queue == other.queue

    def size(self):
        """
        Get the current size of the queue.

        Returns:
            Integer of number of items in queue.
        """

        return len(self.queue)

    def clear(self):
        """Reset queue to empty (no nodes)."""

        self.queue = []

    def top(self):
        """
        Get the top item in the queue.

        Returns:
            The first item stored in the queue.
        """

        return self.queue[0]
    
    def get_nodes(self):
        nodes = []
        for i in range(len(self.queue)):
            nodes.append(self.queue[i][2])
        return nodes

In [74]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestPriorityQueue

TestPriorityQueue("test_append_and_pop").test_append_and_pop(PriorityQueue)
TestPriorityQueue("test_fifo_property").test_fifo_property(PriorityQueue)

UnitTest passed successfully for "test_append_and_pop"!
UnitTest passed successfully for "test_fifo_property"!


### Warmup 2: BFS (5 Pts) <a name="bfs"></a>

To get you started with handling graphs, implement and test breadth-first search over the test network.

You'll complete this by writing the `breadth_first_search()` method. This returns a path of nodes from a given start node to a given end node, as a list.

> **Requirements**
> 1. Implement the `breadth_first_search()` method below.
> 2. If the start and goal are the same, simply return an empty list like `[]`.
> 3. Return the best path found via BFS as a Python list from the start node to the goal node (inclusive).
> 4. You can obtain a dictionary of the neighbors of a node by calling `graph.neighbors(node)`.
> 5. Sort the neighbors of a node alphabetically before adding them to the queue.
> 6. Make sure no node is explored more than once.
> 7. Implement the optimization trick which reduces the number of explored nodes (required for passing the autograder). You may find it useful to re-watch the Canvas videos for this.

> **FAQ**
> 1. **Are there multiple correct paths possible?** No. If you follow the above instructions, each pair of start and goal nodes will result in a unique solution.
> 2. **Do I need Priority Queue for this?** No. BFS does not need to use a priority queue, a simple FIFO queue using a Python `list` should be sufficient. You may use your priority queue implementation if you'd like.
> 3. **Why does it say I don't have the right path when my path cost is shorter?** Remember, BFS treats all edges as having a cost of 1, so the true path cost does not come into play!!
> 4. **I am having trouble reconstructing my optimal path. What should I do?** You can either try to keep track of where you came from and reconstruct the path by chaining parent information, or you can store entire paths in your frontier so that when you find the goal node you can just return the path found without having to do reconstruction.
> 5. **Why does it say my path is invalid?** Did you return the path as a list of nodes including both the start and the goal node? Did you make sure that the path is a valid path and doesn't skip around? (Traverse the graph by hand to make sure it is valid)
> 6. **Why does it say I'm exploring too many nodes?** Did you implement the optimization trick mentioned above? If so, are you making sure that you are not exploring any node more than once? You can check the set of nodes you've explored and how many times they've been explored by calling `graph.explored_nodes`. However please make sure that you remove this call from your code (not comment, fully remove) after you are done debugging as it will cause Gradescope to fail. If you wish to call it repeatedly, make sure you call `graph.reset_search()` to reset the explored count.
> 7. **I'm consistently exploring one too many nodes!** Perhaps you are exploring the goal node when you don't actually need to...

In [ ]:
#export

def breadth_first_search(graph, start, goal):
    """
    Warm-up exercise: Implement breadth-first-search.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """

    if start == goal: return []
    
    bfs_queue = [start]
    visited = set()
    parent_dir = {start: None}
    path = []
    #cost = 0
    
    while len(bfs_queue) > 0:
        current = bfs_queue.pop(0)

        if current == goal:
            #path = []
            while current is not None:
                path.insert(0, current)
                current = parent_dir[current]
            return path
        
        temp_list = []
        
        for neighbor in graph.neighbors(current):
            if neighbor not in visited and neighbor not in parent_dir:
                
                if neighbor == goal:
                    parent_dir[neighbor] = current
                    current = neighbor
                    while current is not None:
                        path.insert(0, current)
                        current = parent_dir[current]
                    return path
                    
                parent_dir[neighbor] = current
                temp_list.append(neighbor)
        
        temp_list = sorted(temp_list)
        #print(temp_list)
        for node in temp_list:
            bfs_queue.append(node)

    return None
        
    #print(graph.neighbors(start))
    #for key in graph.neighbors(start):
      #  print(key)
    
    #print("--------------", type(key), "----------" ,start ,"--------------")
    #raise NotImplementedError

In [ ]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestBFS

TestBFS("test_valid_paths").test_valid_paths(breadth_first_search)
TestBFS("test_optimal_paths").test_optimal_paths(breadth_first_search)
TestBFS("test_explored_counts").test_explored_counts(breadth_first_search)

### Warmup 3: Uniform-cost search (10 Pts) <a name="ucs"></a>

Implement uniform-cost search, using PriorityQueue as your frontier. From now on, PriorityQueue should be your default frontier.

`uniform_cost_search()` should return a path from the start to the goal node as a list of nodes.

The astute of you may have noticed that uniform-cost search is a special case of A*, where the heuristic is the null heuristic, UCS is still different from A* (even though the search behavior will be identical). The difference lies in the fact that UCS is an **uninformed search**, relying solely on path costs, unaware of any heuristics or a node's location. That's why you should **NOT** call A* with a null heuristic, you should **NOT** use heuristics in your implementation, and you should **NOT** be attempting to access a node's position anywhere in your UCS implementation. Doing so will give you a "call(s) to astar detected" error on Gradescope.

> **Requirements**
> 1. Implement the `uniform_cost_search()` method below.
> 2. If the start and goal are the same, simply return an empty list like `[]`.
> 3. Return the best path found as a Python list from the start node to the goal node (inclusive).
> 4. You can obtain a dictionary of the neighbors of a node by calling `graph.neighbors(node)`.
> 5. Sort the neighbors of a node alphabetically before processing them.
> 6. You can access the edge weight between two nodes `u` and `v` by using `graph.get_edge_weight(u, v)`.
> 7. Make sure no node is explored more than once.

In [ ]:
#export

def uniform_cost_search(graph, start, goal):
    """
    Warm-up exercise: Implement uniform_cost_search.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """
    #Check if start == goal, and if return []
    if start == goal:
        return []
    
    #Assign priority queue and add start as first element (root) with priority of 0
    priority_q = PriorityQueue()
    start_tuple = (0, start)
    priority_q.append(start_tuple)
    visited = set()
    parent_dir = {start: ["stop", 0]}
    path = []
    
    #While top in our priority queue is not equal to the goal node
    while (priority_q.top()[2] != goal):
        
        #Assign current node that is being explored, pop it off the priority queue
        #Initialize temporary list = []
        current = priority_q.pop()
        temp_list = []
        
        #Loop through neighbors of current that are not in visited
        for neighbor in graph.neighbors(current[1]):
            
            #Save them to a temporary list of tuples with 
            #edge cost from current to neighbor + total edge cost to current
            
            if neighbor not in visited:
                
                total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parent_dir[current[1]][1]
                #Make current the parent of neighbor if not in parent_dir or smaller than existing entry
                if neighbor not in parent_dir:
                    
                    #Make current the parent of neighbor
                    parent_dir[neighbor] = [current[1], total_edge_cost]
                    
                    temp_cost = total_edge_cost
                    temp_list.append((temp_cost, neighbor))
                    
                elif total_edge_cost < parent_dir[neighbor][1]:
                    
                    parent_dir[neighbor] = [current[1], total_edge_cost]
                    
                    #edge_diff = parent_dir[neighbor][1] - total_edge_cost
                    priority_q.update(neighbor, total_edge_cost)
        
        #TEST SET UP - START
        
        #print("Euclidean distance for p is: ", heuristic(graph, 'p','f'))
        #print("Euclidean distance for r is: ", heuristic(graph, 'r','f'))
        #print("Euclidean distance for s is: ", heuristic(graph, 's','f'))
        #print("Euclidean distance for f is: ", heuristic(graph, 'f','f'))
        #print("Euclidean distance for b is: ", heuristic(graph, 'b','f'))
        #print("Euclidean distance for c is: ", heuristic(graph, 'c','f'))
        
        #TEST SET UP - END
                
        
        #Mark current as visited 
        visited.add(current[1])
        
        sort_and_append(priority_q, temp_list)
        #Sort the list alphabetically based on their tuple[1] value which will be a single character type string
        #sorted_tuples = sorted(temp_list, key=lambda x: x[1])
        
        #Append the list values to the priority queue, which will handle the prioritization
        #for element in sorted_tuples:
            #priority_q.append(element)
            
        #If top of priority queue == goal
        #backtrack from goal node to start using parent_directory and build path, return path
        if priority_q.top()[2] == goal:
            back_track = goal
            
            while back_track != "stop":
                path.insert(0, back_track)
                back_track = parent_dir[back_track][0]
                
            return path

    # TODO: finish this function!͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    #raise NotImplementedError
    
def sort_and_append(q_2_append_2, list_2_append):
    sorted_tuples = sorted(list_2_append, key=lambda x: x[1]) #Sort alphabetically
    for element in sorted_tuples: #Append to priority queue
        q_2_append_2.append(element)

In [ ]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestSearchAlgorithms

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_ucs_romania", uniform_cost_search)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_ucs_romania", uniform_cost_search)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_ucs_romania", uniform_cost_search)

### Warmup 4: A* search (10 Pts) <a name="astar"></a>

Implement A* search using Euclidean distance as your heuristic. You'll need to implement `euclidean_dist_heuristic()` then pass that function to `a_star()` as the heuristic parameter. We provide `null_heuristic()` as a baseline heuristic to test against when calling a_star tests.

The astute of you may have noticed that UCS is a special case of A\*. Perhaps it may save you some time if you implement A* first, then transfer your code over to UCS, and then make the necessary modifications to make the code reflect the UCS requirements...

> **Requirements**
> 1. Implement the `euclidean_dist_heuristic()` and the `a_star()` method below.
> 2. If the start and goal are the same, simply return an empty list like `[]`.
> 3. Return the best path found as a Python list from the start node to the goal node (inclusive).
> 4. You can obtain a dictionary of the neighbors of a node by calling `graph.neighbors(node)`.
> 5. Sort the neighbors of a node alphabetically before processing them.
> 6. You can access the edge weight between two nodes `u` and `v` by using `graph.get_edge_weight(u, v)`.
> 7. You can access the `(x, y)` position of a node `u` by using `graph.nodes[u]['pos']`. You may find this useful for calculating the Euclidean heuristic.
> 8. Make sure no node is explored more than once.

#### The Heuristics

In [89]:
#export

def null_heuristic(graph, v, goal):
    """
    Null heuristic used as a base line.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        v (str): Key for the node to calculate from.
        goal (str): Key for the end node to calculate to.

    Returns:
        0
    """

    return 0


def euclidean_dist_heuristic(graph, u, v):
    """
    Warm-up exercise: Implement the euclidean distance heuristic.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        u (str): Key for the first node to calculate from.
        v (str): Key for the second node to calculate to.

    Returns:
        Euclidean distance between the `u` node and the `v` node
        Round the result to 3 decimal places (if applicable)
    """

   # print(graph.nodes[u]['pos'])
    x_diff = graph.nodes[u]['pos'][0] - graph.nodes[v]['pos'][0]
    y_diff = graph.nodes[u]['pos'][1] - graph.nodes[v]['pos'][1]
    
    return round(math.sqrt(x_diff**2 + y_diff**2), 3)
    # TODO: finish this function!͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    #raise NotImplementedError

In [90]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestEuclideanHeuristic

TestEuclideanHeuristic("test_euclidean_distance").test_euclidean_distance(euclidean_dist_heuristic)

UnitTest passed successfully for "test_euclidean_distance"!


### The A* Algorithm

In [93]:
#export

def a_star(graph, start, goal, heuristic=euclidean_dist_heuristic):
    """
    Warm-up exercise: Implement A* algorithm.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.
        heuristic: Function to determine distance heuristic.
            Default: euclidean_dist_heuristic.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """
    
    #Check if start == goal, and if return []
    if start == goal:
        return []
    
    #Assign priority queue and add start as first element (root) with priority of 0 + euclidean distance to goal (heuristic)
    priority_q = PriorityQueue()
    start_tuple = (heuristic(graph, start, goal), start)
    priority_q.append(start_tuple)
    visited = set()
    parent_dir = {start: ["stop", 0]}
    path = []
    
    #While top in our priority queue is not equal to the goal node
    while (priority_q.top()[2] != goal):
        
        #Assign current node that is being explored, pop it off the priority queue
        #Initialize temporary list = []
        current = priority_q.pop()
        temp_list = []
        
        #Loop through neighbors of current that are not in visited
        for neighbor in graph.neighbors(current[1]):
            
            #Save them to a temporary list of tuples with 
            #(edge cost from current to neighbor + euclidean distance from neighbor to goal (heuristic), node)
            
            if neighbor not in visited:
                
                total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parent_dir[current[1]][1]
                #Make current the parent of neighbor if not in parent_dir or smaller than existing entry
                if neighbor not in parent_dir:
                    
                    #Make current the parent of neighbor
                    parent_dir[neighbor] = [current[1], total_edge_cost]
                    
                    temp_cost = total_edge_cost + heuristic(graph, neighbor, goal)
                    temp_list.append((temp_cost, neighbor))
                    
                elif total_edge_cost < parent_dir[neighbor][1]:
                    
                    parent_dir[neighbor] = [current[1], total_edge_cost]
                    
                    #edge_diff = parent_dir[neighbor][1] - total_edge_cost
                    priority_q.update(neighbor, total_edge_cost + heuristic(graph, neighbor, goal))
        
        #TEST SET UP - START
        #print("Euclidean distance for p is: ", heuristic(graph, 'p','f'))
        #print("Euclidean distance for r is: ", heuristic(graph, 'r','f'))
        #print("Euclidean distance for s is: ", heuristic(graph, 's','f'))
        #print("Euclidean distance for f is: ", heuristic(graph, 'f','f'))
        #print("Euclidean distance for b is: ", heuristic(graph, 'b','f'))
        #print("Euclidean distance for c is: ", heuristic(graph, 'c','f'))
        #print(priority_q.get_nodes())
        #TEST SET UP - END
                
        
        #Mark current as visited 
        visited.add(current[1])
        
        #Sort the list alphabetically based on their tuple[1] value which will be a single character type string
        sorted_tuples = sorted(temp_list, key=lambda x: x[1])
        
        #Append the list values to the priority queue, which will handle the prioritization
        for element in sorted_tuples:
            priority_q.append(element)
            
        #If top of priority queue == goal
        #backtrack from goal node to start using parent_directory and build path, return path
        if priority_q.top()[2] == goal:
            back_track = goal
            
            while back_track != "stop":
                path.insert(0, back_track)
                back_track = parent_dir[back_track][0]
                
            return path
        
    #raise NotImplementedError

In [94]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestSearchAlgorithms

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_a_star_null_romania", a_star, heuristic=null_heuristic)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_a_star_null_romania", a_star, heuristic=null_heuristic)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_a_star_null_romania", a_star, heuristic=null_heuristic)

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_a_star_euclidean_romania", a_star, heuristic=euclidean_dist_heuristic)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_a_star_euclidean_romania", a_star, heuristic=euclidean_dist_heuristic)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_a_star_euclidean_romania", a_star, heuristic=euclidean_dist_heuristic)

UnitTest passed successfully for "test_a_star_null_romania.test_valid_paths"!
UnitTest passed successfully for "test_a_star_null_romania.test_optimal_paths"!
UnitTest passed successfully for "test_a_star_null_romania.test_explored_counts"!
UnitTest passed successfully for "test_a_star_euclidean_romania.test_valid_paths"!
UnitTest passed successfully for "test_a_star_euclidean_romania.test_optimal_paths"!
UnitTest passed successfully for "test_a_star_euclidean_romania.test_explored_counts"!


## Exercises

The following exercises will require you to implement several kinds of bidirectional and tridirectional searches. The benefits of these algorithms over uninformed or unidirectional search are more clearly seen on larger graphs.

For these exercises, we recommend you take a look at the resources mentioned earlier.

### Exercise 1: Bidirectional uniform-cost search (20 Pts) <a name="bi-ucs"></a>

Implement bidirectional uniform-cost search. Remember that this requires starting your search at both the start and goal nodes.

`bidirectional_ucs()` should return the path from the start node to the goal node (as a list of nodes).

> **Requirements**
> 1. Implement the `bidirectional_ucs()` method below.
> 2. If the start and goal are the same, simply return an empty list like `[]`.
> 3. Return the best path found as a Python list from the start node to the goal node (inclusive).
> 4. You can obtain a dictionary of the neighbors of a node by calling `graph.neighbors(node)`.
> 5. Sort the neighbors of a node alphabetically before processing them.
> 6. You can access the edge weight between two nodes `u` and `v` by using `graph.get_edge_weight(u, v)`.
> 7. Make sure no node is explored more than once, regardless of which frontier explored it.
> 8. We provide a small margin of error for your explored count when grading.

> **Hints**
> 1. Your optimal path should be the same optimal path found via unidirectional UCS and A*.
> 2. Which frontier should you pop first (forwards or backwards)? There are two common approaches, consider trying both to see which one performs better.
> 3. When should I stop so that I don't explore too many nodes? We love the resource titled: [Bi Directional Stopping Conditions, Piazza '17](https://docs.google.com/document/d/14Wr2SeRKDXFGdD-qNrBpXjW8INCGIfiAoJ0UkZaLWto/pub)
> 4. Think about what statement you can make about the relationship between the length of the path to a node you've just popped off the frontier and the length of the paths that are still on the frontier, as you think about your stopping condition.

In [86]:
#export

def bidirectional_ucs(graph, start, goal):
    """
    Exercise 1: Bidirectional Search.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """
    
    #If start == goal, return [], elif start is neighbor of goal, return [start, goal]
    if start == goal: return []
    
    #Initialize 2 priority qs and visited lists, forward and backward
    q_forward, q_backward = PriorityQueue(), PriorityQueue()
    visited_f, visited_b = set(), set()
    forward = True
    
    q_forward.append((0, start))
    q_backward.append((0, goal))
    
    #Initialize parent directories
    parents_f = {start: ["stop", 0]}
    parents_b = {goal: ["stop", 0]}
    
    # SET UP COMPLETE - NOW TO THE GOOD STUFF
    
    while (not visited_f.intersection(visited_b)): #while explored sets from forward and backward don't intersect
        #Forward
        if forward:   
            current = q_forward.pop()
            temp_list = []      
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in visited_f:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_f[current[1]][1]
                    if neighbor not in parents_f: #If neighbor has no parent yet
                        parents_f[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_list.append((total_edge_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_f[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_f[neighbor] = [current[1], total_edge_cost] #Update parent
                        q_forward.update(neighbor, total_edge_cost) #Update queue
            
            visited_f.add(current[1]) #Add to forward set of visited
            sort_and_append(q_forward, temp_list) #Sort list alphabetically and append to forward queue
            
            forward = False
        
        #Backward 
        else: 
            current = q_backward.pop()
            temp_list = []      
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in visited_b:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_b[current[1]][1]
                    if neighbor not in parents_b: #If neighbor has no parent yet
                        parents_b[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_list.append((total_edge_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_b[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_b[neighbor] = [current[1], total_edge_cost] #Update parent
                        q_backward.update(neighbor, total_edge_cost) #Update queue
            
            visited_b.add(current[1]) #Add to forward set of visited
            sort_and_append(q_backward, temp_list) #Sort list alphabetically and append to forward queue
        
            forward = True
            
        #print(visited_b, visited_f)
        
    #As while loop stopped, explored sets intersect on one element
    intersection_node = next(iter(visited_f.intersection(visited_b))) #Find intersection node
    
    ###
    crossover_points = list(visited_f.intersection(set((q_backward.get_nodes() + list(visited_b)))))
    #print("Intersection node: ", intersection_node)
    #print("test crossover fct input, should be set", set((q_backward.get_nodes() + list(visited_b))))
    #print("see crossover points: ",crossover_points)

    path_costs = []
    for point in crossover_points:
        path_costs.append([point, parents_b[point][1] + parents_f[point][1]])
        
    path_costs = sorted(path_costs, key=lambda x: x[1])
    #path_costs = [sublist for sublist in path_costs if intersection_node not in sublist[0]]
    path_costs = [sublist for sublist in path_costs if intersection_node != sublist[0]]
    #print("See sorted path costs list: ", path_costs)
    
    intersection_cost = parents_b[intersection_node][1] + parents_f[intersection_node][1]
    #print("intersection cost: ", intersection_cost)
    #print("Path cost: ", path_costs[0][1])
    
    if intersection_cost > path_costs[0][1]:
        #print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        forward_path = backtrack_and_append(path_costs[0][0], parents_f)
        backward_path = backtrack_and_append(path_costs[0][0], parents_b)
    
        backward_path = backward_path[:-1] #Remove last element
        backward_path.reverse()
    
        path = forward_path + backward_path
        
        return path
    
    
    forward_path = backtrack_and_append(intersection_node, parents_f)
    backward_path = backtrack_and_append(intersection_node, parents_b)
    
    backward_path = backward_path[:-1] #Remove last element
    backward_path.reverse()
    
    path = forward_path + backward_path
    
    #Once intersection is found, assign upper bound to find an alternative intersection point to be sum of edge costs
    #from the parent of the intersection node for forward and backward seearch to the intersection node
    #The lower bound will be not important but the edge cost form parent of intersection to intersection 
    #for the respective search.
    
    #Save total path cost to be sum of total edge cost to intersection for both searches
    
    #Find all potential crossover points to be the intersection of nodes between the set of 
    #forward explroed (visited_f and) 
    #the union of backward explored (visited_b) and backsearch frontier (q_backward)
    
    #Calculate the total path cost for each crossover point = sum(cost to this point from forward and backward)
    
    #For the lowest cost crossover point, explore it from the backward frontier
    
    return path
    
    #raise NotImplementedError
    
def sort_and_append(q_2_append_2, list_2_append):
    sorted_tuples = sorted(list_2_append, key=lambda x: x[1]) #Sort alphabetically
    for element in sorted_tuples: #Append to priority queue
        q_2_append_2.append(element)

def backtrack_and_append(intersection_node, parent_directory):
    path = []
    back_track = intersection_node
    while back_track != "stop":
        path.insert(0, back_track)
        back_track = parent_directory[back_track][0]
    return path

In [87]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestSearchAlgorithms

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_bi_ucs_romania", bidirectional_ucs)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_bi_ucs_romania", bidirectional_ucs)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_bi_ucs_romania", bidirectional_ucs)

UnitTest passed successfully for "test_bi_ucs_romania.test_valid_paths"!
UnitTest passed successfully for "test_bi_ucs_romania.test_optimal_paths"!
UnitTest passed successfully for "test_bi_ucs_romania.test_explored_counts"!


### Exercise 2: Bidirectional A* search (29 Pts) <a name="bi-astar"></a>

Implement bidirectional A* search. Remember that you need to calculate a heuristic for both the start-to-goal search and the goal-to-start search.

> **Requirements**
> 1. Implement the `bidirectional_a_star()` method below.
> 2. If the start and goal are the same, simply return an empty list like `[]`.
> 3. Return the best path found as a Python list from the start node to the goal node (inclusive).
> 4. You can obtain a dictionary of the neighbors of a node by calling `graph.neighbors(node)`.
> 5. Sort the neighbors of a node alphabetically before processing them.
> 6. You can access the edge weight between two nodes `u` and `v` by using `graph.get_edge_weight(u, v)`.
> 7. Make sure no node is explored more than once, regardless of which frontier explored it.
> 8. We provide a small margin of error for your explored count when grading.

> **Hints**
> 1. Your optimal path should be the same optimal path found via unidirectional UCS and A*.
> 2. Which frontier should you pop first (forwards or backwards)? There are two common approaches, consider trying both to see which one performs better.
> 3. When should I stop so that I don't explore too many nodes? We love the resource titled: [Bi Directional Stopping Conditions, Piazza '17](https://docs.google.com/document/d/14Wr2SeRKDXFGdD-qNrBpXjW8INCGIfiAoJ0UkZaLWto/pub)
> 4. Think about what statement you can make about the relationship between the length of the path to a node you've just popped off the frontier and the length of the paths that are still on the frontier, as you think about your stopping condition.
> 5. How different is the stopping condition from bidirectional UCS? Perhaps there is some accelerated stopping condition that works for both Bi-UCS and Bi-A*...

In [338]:
#export

def bidirectional_a_star(graph, start, goal,
                         heuristic=euclidean_dist_heuristic):
    """
    Exercise 2: Bidirectional A*.

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.
        heuristic: Function to determine distance heuristic.
            Default: euclidean_dist_heuristic.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """
    #Start condition and initializations
    if start == goal: return []

    q_forward, q_backward = PriorityQueue(), PriorityQueue()
    visited_f, visited_b = set(), set()
    forward = True
    q_forward.append((0, start))
    q_backward.append((0, goal))
    parents_f = {start: ["stop", 0]}
    parents_b = {goal: ["stop", 0]}
    
    while (not visited_f.intersection(visited_b)): #while explored sets from forward and backward don't intersect
        
        if forward:   
            current = q_forward.pop()
            temp_list = []      
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in visited_f:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_f[current[1]][1]
                    if neighbor not in parents_f: #If neighbor has no parent yet
                        parents_f[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_cost = total_edge_cost + heuristic(graph, neighbor, goal)
                        temp_list.append((temp_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_f[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_f[neighbor] = [current[1], total_edge_cost] #Update parent
                        q_forward.update(neighbor, temp_cost) #Update queue
            
            visited_f.add(current[1]) #Add to forward set of visited
            sort_and_append(q_forward, temp_list) #Sort list alphabetically and append to forward queue
            
            forward = False
        
        #Backward 
        else: 
            current = q_backward.pop()
            temp_list = []      
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in visited_b:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_b[current[1]][1]
                    if neighbor not in parents_b: #If neighbor has no parent yet
                        parents_b[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_cost = total_edge_cost + heuristic(graph, neighbor, start)
                        temp_list.append((temp_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_b[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_b[neighbor] = [current[1], total_edge_cost] #Update parent
                        q_backward.update(neighbor, temp_cost) #Update queue
            
            visited_b.add(current[1]) #Add to forward set of visited
            sort_and_append(q_backward, temp_list) #Sort list alphabetically and append to forward queue
        
            forward = True
            
        #print(visited_b, visited_f)
        
    #As while loop stopped, explored sets intersect on one element
    intersection_node = next(iter(visited_f.intersection(visited_b))) #Find intersection node
    
    ###
    crossover_points = list(visited_f.intersection(set((q_backward.get_nodes() + list(visited_b)))))
    #print("Intersection node: ", intersection_node)
    #print("test crossover fct input, should be set", set((q_backward.get_nodes() + list(visited_b))))
    #print("see crossover points: ",crossover_points)

    path_costs = []
    for point in crossover_points:
        path_costs.append([point, parents_b[point][1] + parents_f[point][1]])
        
    path_costs = sorted(path_costs, key=lambda x: x[1])
    #path_costs = [sublist for sublist in path_costs if intersection_node not in sublist[0]]
    path_costs = [sublist for sublist in path_costs if intersection_node != sublist[0]]
    #print("See sorted path costs list: ", path_costs)
    
    intersection_cost = parents_b[intersection_node][1] + parents_f[intersection_node][1]
    #print("intersection cost: ", intersection_cost)
    #print("Path cost: ", path_costs[0][1])
    
    if intersection_cost > path_costs[0][1]:
        #print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        forward_path = backtrack_and_append(path_costs[0][0], parents_f)
        backward_path = backtrack_and_append(path_costs[0][0], parents_b)
    
        backward_path = backward_path[:-1] #Remove last element
        backward_path.reverse()
    
        path = forward_path + backward_path
        
        return path
    
    
    forward_path = backtrack_and_append(intersection_node, parents_f)
    backward_path = backtrack_and_append(intersection_node, parents_b)
    
    backward_path = backward_path[:-1] #Remove last element
    backward_path.reverse()
    
    path = forward_path + backward_path
    
    #Once intersection is found, assign upper bound to find an alternative intersection point to be sum of edge costs
    #from the parent of the intersection node for forward and backward seearch to the intersection node
    #The lower bound will be not important but the edge cost form parent of intersection to intersection 
    #for the respective search.
    
    #Save total path cost to be sum of total edge cost to intersection for both searches
    
    #Find all potential crossover points to be the intersection of nodes between the set of 
    #forward explroed (visited_f and) 
    #the union of backward explored (visited_b) and backsearch frontier (q_backward)
    
    #Calculate the total path cost for each crossover point = sum(cost to this point from forward and backward)
    
    #For the lowest cost crossover point, explore it from the backward frontier
    
    return path
    
    #raise NotImplementedError
    
def sort_and_append(q_2_append_2, list_2_append):
    sorted_tuples = sorted(list_2_append, key=lambda x: x[1]) #Sort alphabetically
    for element in sorted_tuples: #Append to priority queue
        q_2_append_2.append(element)

def backtrack_and_append(intersection_node, parent_directory):
    path = []
    back_track = intersection_node
    while back_track != "stop":
        path.insert(0, back_track)
        back_track = parent_directory[back_track][0]
    return path
    # TODO: finish this function!͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    #raise NotImplementedError

In [339]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestSearchAlgorithms

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_bi_a_star_null_romania", bidirectional_a_star, heuristic=null_heuristic)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_bi_a_star_null_romania", bidirectional_a_star, heuristic=null_heuristic)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_bi_a_star_null_romania", bidirectional_a_star, heuristic=null_heuristic)

TestSearchAlgorithms("test_valid_paths").test_valid_paths("test_bi_a_star_euclidean_romania", bidirectional_a_star, heuristic=euclidean_dist_heuristic)
TestSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_bi_a_star_euclidean_romania", bidirectional_a_star, heuristic=euclidean_dist_heuristic)
TestSearchAlgorithms("test_explored_counts").test_explored_counts("test_bi_a_star_euclidean_romania", bidirectional_a_star, heuristic=euclidean_dist_heuristic)

UnitTest passed successfully for "test_bi_a_star_null_romania.test_valid_paths"!
UnitTest passed successfully for "test_bi_a_star_null_romania.test_optimal_paths"!
UnitTest passed successfully for "test_bi_a_star_null_romania.test_explored_counts"!
UnitTest passed successfully for "test_bi_a_star_euclidean_romania.test_valid_paths"!
UnitTest passed successfully for "test_bi_a_star_euclidean_romania.test_optimal_paths"!
UnitTest passed successfully for "test_bi_a_star_euclidean_romania.test_explored_counts"!


### Exercise 3: Tridirectional UCS search (12 Pts) <a name="tri-ucs"></a>

Implement tridirectional search using UCS. Starting from each goal node, perform a uniform-cost search and keep expanding until two of the three searches meet. This should create one continuous path that connects all three nodes. You can return the path in any order. Eg. (1->2->3 == 3->2->1). You may also want to look at the Tri-city search challenge question on Canvas.

For example, suppose we have goal nodes `[a,b,c]`. Then what we want you to do is to start at node `a` and expand like in a normal search. However, notice that you will be searching for both nodes `b` and `c` during this search and a similar search will also start from nodes `b` and `c`.

**This is not the same as 3 bi-directional searches,** as that would result in 6 frontiers!!

> **Requirements**
> 1. Implement the `tridirectional_search()` method below.
> 2. If all three goals are the same, simply return an empty list like `[]`.
> 3. If there are 2 identical goals (i.e. `[a,b,b]`) then return the path `[a...b]` (i.e. just the path from `a` to `b`). Do **NOT** call your bidirectional searches for this, your tridirectional search should naturally handle this scenario.
> 4. Return the best path found as a Python list connecting all three goal nodes (inclusive).
> 5. You can obtain a dictionary of the neighbors of a node by calling `graph.neighbors(node)`.
> 6. Sort the neighbors of a node alphabetically before processing them.
> 7. You can access the edge weight between two nodes `u` and `v` by using `graph.get_edge_weight(u, v)`.
> 8. Make sure no node is explored more than once, regardless of which frontier explored it.
> 9. We provide a small margin of error for your explored count when grading.

> **Hints**
> 1. **Why can't I just run three bidirectional searches?** If you run three bidirectional searches, you aren’t taking advantage of the team knowledge collected by all the searches, and end up exploring multiple nodes, multiple times, blowing up your explored count. This is a team effort, all three goals should be searching for each other together, sharing information as necessary to prevent exploration of any node more than once.
> 2. **Why can’t I just take the first two paths I find and call it a day?** Are you sure that’s guaranteed to be the most optimal path and the third path found is guaranteed to be larger than the two paths you’ve found? This could vary based on your implementation.
> 3. **Why can’t I just find all three paths and then take the two shortest ones?** Are you sure you HAVE to find all three paths in order to pick the two shortest ones? You might end up exploring too many nodes in some cases…
> 4. **Ok if I can’t do either of the above then what can I even do?** Is it really black and white? Is there really no in-between option?
> 5. **How is it possible not to explore any node more than once when there are two searches?** The search is a team effort. All teammates are aware of each other’s knowledge so no node should ever have to be explored more than once.
> 6. **Why are there no provided resources?** This is a thinking exercise. You are meant to build upon what you have learned so far from the assignment and to think about what a Tridirectional search brings to the table and how you can deal with those new issues and tricks. Homework isn’t about us giving you the algorithm and you just implementing it in code, there’s no value there really, then it just becomes a test of your Python knowledge. The value here comes from your ability to take what you have learned and take it one step further to build something entirely new and advanced. This is where the value of this assignment lies.
> 7. Try creating small graph scenarios on paper to see what interesting situations may arise. This will help you as you try to develop your stopping condition(s).

![romania.png](romania/romania.png)

Who met:  ('a', '2', '3', 445)  
Q Continue before:  PQ:[[198, 8, 'r'], [211, 1, 'f'], [227, 6, 'v'], [239, 7, 'c'], [269, 9, 'e']]  
Intersect with 3rd before:  c  
Q Contiune after:  PQ:[[239, 7, 'c'], [269, 9, 'e'], [278, 10, 's'], [319, 11, 'i']]  
Intersect with 3rd after:  c  
Shortest path:  434  
Q Continue Top:  c  
V2:  {'o', 'a', 's', 'z'}  
Cost:  239  
Shortest Path:  434  
Q Continue Top:  e  
V2:  {'o', 'a', 's', 'z'}  
Cost:  269  
Shortest Path:  434  
Q Continue Top:  s  
V2:  {'o', 'a', 's', 'z'}  
Cost:  278  
Shortest Path:  434  
Node after update:  s   

Actual: ['b', 'p', 'r', 's', 'o', 'z', 'a', 't', 'l', 'm']  
Goal nodes ('b', 'm', 'o')   
Expected path is ['o', 's', 'r', 'p', 'b', 'p', 'c', 'd', 'm']  

In [464]:
#export

def tridirectional_search(graph, goals):
    
    ### SET UP 
    if goals[0] == goals[1] == goals[2]: return [] 
    #test = False
    #if goals == ['b', 'm', 'o']:
        #test = True
    q1, q2, q3 = PriorityQueue(), PriorityQueue(), PriorityQueue()
    v1, v2, v3 = set(), set(), set()
    one, two, no_need_to_compare = True, True, False
    
    q1.append((0, goals[0]))
    q2.append((0, goals[1]))
    q3.append((0, goals[2]))

    parents_1 = {goals[0]: ["stop", 0]}
    parents_2 = {goals[1]: ["stop", 0]}
    parents_3 = {goals[2]: ["stop", 0]}
    
    ###############################
    
    ### INITIAL LOOP TO FIND FIRST INTERSECTION
    
    while ((q1.top()[2] not in v1.union(v2, v3)) and (q2.top()[2] not in v1.union(v2, v3)) and 
           (q3.top()[2] not in v1.union(v2, v3))):
        
        if one: #Search 1
            current = q1.pop()
            temp_list = []   
            
            
            if q1.size() != 0:
                if q1.top()[2] in v1.union(v2, v3):
                    v1.add(current[1]) #Add to forward set of visited
                    break
            
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in v1:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_1[current[1]][1]
                    if neighbor not in parents_1: #If neighbor has no parent yet
                        parents_1[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_list.append((total_edge_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_1[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_1[neighbor] = [current[1], total_edge_cost] #Update parent
                        q1.update(neighbor, total_edge_cost) #Update queue 
            v1.add(current[1]) #Add to forward set of visited
            sort_and_append(q1, temp_list) #Sort list alphabetically and append to queue
            one = False
        elif two: #Search 2 
            current = q2.pop()
            temp_list = []   
            
            if q2.size() != 0:
                if q2.top()[2] in v1.union(v2, v3):
                    v2.add(current[1]) #Add to forward set of visited
                    break
                    
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in v2:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_2[current[1]][1]
                    if neighbor not in parents_2: #If neighbor has no parent yet
                        parents_2[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_list.append((total_edge_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_2[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_2[neighbor] = [current[1], total_edge_cost] #Update parent
                        q2.update(neighbor, total_edge_cost) #Update queue
            v2.add(current[1]) #Add to forward set of visited
            sort_and_append(q2, temp_list) #Sort list alphabetically and append to queue
            two = False
        else: #Search 3 
            current = q3.pop()
            temp_list = []    
            
            if q3.size() != 0:
                if q3.top()[2] in v1.union(v2, v3):
                    v3.add(current[1]) #Add to forward set of visited
                    break
                    
            for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
                if neighbor not in v3:
                    total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_3[current[1]][1]
                    if neighbor not in parents_3: #If neighbor has no parent yet
                        parents_3[neighbor] = [current[1], total_edge_cost] #Assign parent
                        temp_list.append((total_edge_cost, neighbor)) #Append to list    
                    elif total_edge_cost < parents_3[neighbor][1]: #If parent exists, but we found a cheaper path
                        parents_3[neighbor] = [current[1], total_edge_cost] #Update parent
                        q3.update(neighbor, total_edge_cost) #Update queue
            v3.add(current[1]) #Add to forward set of visited
            sort_and_append(q3, temp_list) #Sort list alphabetically and append to queue
            one = True 
            two = True       
    ####################################
        
    ### OPTIMIZE FIRST PATH AB
    met = who_met_3(q1, q2, q3, v1, v2, v3, parents_1, parents_2, parents_3)
    eval('v' + str(met[1])).add(met[0])
    eval('v' + str(met[2])).add(met[0])
    ab_intersect = met[0]
    crossover_points = list(eval('v' + str(met[1])).intersection(set((eval('q' + str(met[2])).get_nodes() + 
                                                                             list(eval('v' + str(met[2])))))))
    path_costs = []
    for point in crossover_points:
        path_costs.append([point, eval('parents_' + str(met[2]))[point][1] + 
                           eval('parents_' + str(met[1]))[point][1]])
    path_costs = sorted(path_costs, key=lambda x: x[1])
    path_costs = [sublist for sublist in path_costs if ab_intersect != sublist[0]]
    intersection_cost = eval('parents_' + str(met[2]))[met[0]][1] + eval('parents_' + str(met[1]))[met[0]][1]
    if len(path_costs) != 0:
        if intersection_cost > path_costs[0][1]:
            ab_intersect = path_costs[0][0]
            eval('v' + str(met[1])).add(ab_intersect)
            eval('v' + str(met[2])).add(ab_intersect)
    ab_path_cost = eval('parents_' + str(met[2]))[ab_intersect][1] + eval('parents_' + str(met[1]))[ab_intersect][1]
    #####################################
    
    ### Continue Search with 3rd Search I
    temp1, temp2 = [met[1], met[2]], ['1', '2','3']
    result = [item for item in temp2 if item not in temp1] #result contains leftover search number
    q_continue = eval('q' + str(result[0]))
    parents_continue = eval('parents_' + str(result[0]))

    while (q_continue.top()[2] not in eval('v' + str(met[1])).union(eval('v' + str(met[2])))):
        current = q_continue.pop()
        temp_list = [] 
        """
        if q_continue.size() != 0:
            if q_continue.top()[2] in eval('v' + str(met[1])).union(eval('v' + str(met[2]))):
                sort_and_append(q_continue, temp_list)
                break
         """
        for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
            total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_continue[current[1]][1]
            if neighbor not in parents_continue: #If neighbor has no parent yet
                parents_continue[neighbor] = [current[1], total_edge_cost] #Assign parent
                temp_list.append((total_edge_cost, neighbor)) #Append to list    
            elif total_edge_cost < parents_continue[neighbor][1]: #If parent exists, but we found a cheaper path
                parents_continue[neighbor] = [current[1], total_edge_cost] #Update parent
                q_continue.update(neighbor, total_edge_cost) #Update queue               
        sort_and_append(q_continue, temp_list) #Sort list alphabetically and append to queue
    #####################################
    
    ### OPTIMIZE SECOND PATH C - 
    c_search_x_initial = who_met_2(q_continue.top()[2], eval('v' + str(met[1])), met[1], eval('v' + str(met[2])), met[2])  
    crossover_points = list(eval('v' + str(c_search_x_initial)).intersection(set((q_continue.get_nodes()))))
    path_costs = []
    for point in crossover_points:
        path_costs.append([point, parents_continue[point][1] + eval('parents_' + str(c_search_x_initial))[point][1]])    
    path_costs = sorted(path_costs, key=lambda x: x[1])
    path_costs = [sublist for sublist in path_costs if q_continue.top()[2] != sublist[0]]
    intersection_cost = parents_continue[q_continue.top()[2]][1] + eval('parents_' + str(c_search_x_initial))[q_continue.top()[2]][1]
    intersect_node_c_x_initial = q_continue.top()[2]
    if len(path_costs) != 0:
        if intersection_cost > path_costs[0][1]:
            intersect_node_c_x_initial = path_costs[0][0] #Update intersect node 
    c_x_path_cost_1 = parents_continue[intersect_node_c_x_initial][1] + eval('parents_' + str(c_search_x_initial))[intersect_node_c_x_initial][1]
    ######################################

    ### Continue Search with 3rd Search II
    options = [met[1], met[2]]
    c_search_x_second = [item for item in options if item != c_search_x_initial][0]
    
    if q_continue.top()[0] >= max(c_x_path_cost_1, ab_path_cost):
        no_need_to_compare = True
        
    while (q_continue.top()[2] not in eval('v' + str(c_search_x_second))) and not no_need_to_compare:
        if q_continue.size() == 1:
            if q_continue.top()[0] >= max(c_x_path_cost_1, ab_path_cost):
                break
                
        current = q_continue.pop()
        temp_list = []      
        
        if q_continue.size() != 0:
            if q_continue.top()[0] >= max(c_x_path_cost_1, ab_path_cost):
                break
                
        for neighbor in graph.neighbors(current[1]): #Loop through neighbors of current that are not in visited
            total_edge_cost = graph.get_edge_weight(current[1], neighbor) + parents_continue[current[1]][1]
            if neighbor not in parents_continue: #If neighbor has no parent yet
                parents_continue[neighbor] = [current[1], total_edge_cost] #Assign parent
                temp_list.append((total_edge_cost, neighbor)) #Append to list    
            elif total_edge_cost < parents_continue[neighbor][1]: #If parent exists, but we found a cheaper path
                parents_continue[neighbor] = [current[1], total_edge_cost] #Update parent
                q_continue.update(neighbor, total_edge_cost) #Update queue                 
        sort_and_append(q_continue, temp_list) #Sort list alphabetically and append to queue    
        
    if q_continue.top()[0] >= max(c_x_path_cost_1, ab_path_cost):
        no_need_to_compare = True
    #######################################
    
    ### OPTIMIZE THIRD PATH C - (if applicable)
    if not no_need_to_compare:
        crossover_points = list(eval('v' + str(c_search_x_second)).intersection(set((q_continue.get_nodes()))))
        path_costs = []
        intersect_node_c_x_second = q_continue.top()[2]
        for point in crossover_points:
            path_costs.append([point, parents_continue[point][1] + eval('parents_' + str(c_search_x_second))[point][1]])    
        path_costs = sorted(path_costs, key=lambda x: x[1])
        path_costs = [sublist for sublist in path_costs if intersect_node_c_x_second != sublist[0]]
        if not no_need_to_compare:
            intersection_cost = parents_continue[intersect_node_c_x_second][1] + eval('parents_' + str(c_search_x_second))[intersect_node_c_x_second][1]
            if len(path_costs) != 0:
                if intersection_cost > path_costs[0][1]:
                    intersect_node_c_x_second = path_costs[0][0] #Update intersect node 
            c_x_path_cost_2 = parents_continue[intersect_node_c_x_second][1] + eval('parents_' + str(c_search_x_second))[intersect_node_c_x_second][1]
    ######################################
    
    ### Build Path
    #Case 1: CB was determined to be longer than max of (AB, AC) and therefore never searched for
    if no_need_to_compare:
        c_path = backtrack_and_append(intersect_node_c_x_initial, parents_continue)
        temp_path = backtrack_and_append(intersect_node_c_x_initial, eval('parents_' + str(c_search_x_initial)))
        temp_path.reverse()
        c_path = c_path + temp_path
        if c_search_x_initial == met[1]:
            ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
            b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
            b_path.reverse()
            ab_path = ab_path + b_path
        else: 
            ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
            b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
            b_path.reverse()
            ab_path = ab_path + b_path
            
        full_path = c_path + ab_path
        result_path = filter_nodes(full_path)
        return result_path
    
    #Case 2: CB was found. If CA <= CB
    if c_x_path_cost_1 <= c_x_path_cost_2:
        c_path = backtrack_and_append(intersect_node_c_x_initial, parents_continue)
        temp_path = backtrack_and_append(intersect_node_c_x_initial, eval('parents_' + str(c_search_x_initial)))
        temp_path.reverse()
        c_path_1 = c_path + temp_path
        if (c_x_path_cost_2 < ab_path_cost) and (c_x_path_cost_1 != c_x_path_cost_2):
            temp1 = backtrack_and_append(intersect_node_c_x_second, eval('parents_' + str(c_search_x_second)))
            temp2 = backtrack_and_append(intersect_node_c_x_second, parents_continue)
            temp2.reverse()
            c_path_2 = temp1 + temp2
            full_path = c_path_2 + c_path_1
            result_path = filter_nodes(full_path)
            return result_path
        else:
            if c_search_x_initial == met[1]:
                ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
                b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
                b_path.reverse()
                ab_path = ab_path + b_path
            else: 
                ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
                b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
                b_path.reverse()
                ab_path = ab_path + b_path
            full_path = c_path_1 + ab_path
            result_path = filter_nodes(full_path)
            return result_path
    
    #Case 3: CB was found. If CA > CB
    c_path = backtrack_and_append(intersect_node_c_x_second, parents_continue)
    temp_path = backtrack_and_append(intersect_node_c_x_second, eval('parents_' + str(c_search_x_second)))
    temp_path.reverse()
    c_path = c_path + temp_path
    if (c_x_path_cost_1 < ab_path_cost) and (c_x_path_cost_1 != c_x_path_cost_2):
        temp1 = backtrack_and_append(intersect_node_c_x_initial, eval('parents_' + str(c_search_x_initial)))
        temp2 = backtrack_and_append(intersect_node_c_x_initial, parents_continue)
        temp2.reverse()
        c_path_2 = temp1 + temp2
        full_path = c_path_2 + c_path
        result_path = filter_nodes(full_path)
        return result_path
    else:
        if c_search_x_second == met[1]:
            ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
            b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
            b_path.reverse()
            ab_path = ab_path + b_path
        else: 
            ab_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[2])))
            b_path = backtrack_and_append(ab_intersect, eval('parents_' + str(met[1])))
            b_path.reverse()
            ab_path = ab_path + b_path
        
        full_path = c_path + ab_path
        result_path = filter_nodes(full_path)
        return result_path
        
##########################################  
##########################################  
##########################################  
def filter_nodes(full_path):
    result = []
    for i in range(len(full_path) - 1):
        if full_path[i] != full_path[i + 1]:
            result.append(full_path[i])
    result.append(full_path[-1])
    return result
def sort_and_append(q_2_append_2, list_2_append):
    sorted_tuples = sorted(list_2_append, key=lambda x: x[1]) #Sort alphabetically
    for element in sorted_tuples: #Append to priority queue
        q_2_append_2.append(element)

def backtrack_and_append(intersection_node, parent_directory):
    path, counter = [], 0
    back_track = intersection_node
    while back_track != "stop":
        path.insert(0, back_track)
        back_track = parent_directory[back_track][0]
    return path
    
def who_met_2(q1, v2,num2, v3,num3):
    if q1 in v2:
        return str(num2)
    elif q1 in v3:
        return str(num3)
    return None

def who_met_3(q1, q2, q3, v1, v2, v3, p1, p2, p3):
    intersections = []
    if q1.top()[2] in v2:
        intersections.append((q1.top()[2], '1', '2', p1[q1.top()[2]][1] + p2[q1.top()[2]][1]))
    if q1.top()[2] in v3:
        intersections.append((q1.top()[2], '1', '3', p1[q1.top()[2]][1] + p3[q1.top()[2]][1]))
    if q2.top()[2] in v1:
        intersections.append((q2.top()[2], '2', '1', p2[q2.top()[2]][1] + p1[q2.top()[2]][1]))
    if q2.top()[2] in v3:
        intersections.append((q2.top()[2], '2', '3', p2[q2.top()[2]][1] + p3[q2.top()[2]][1]))
    if q3.top()[2] in v1:
        intersections.append((q3.top()[2], '3', '1', p3[q3.top()[2]][1] + p1[q3.top()[2]][1]))
    if q3.top()[2] in v2:
        intersections.append((q3.top()[2], '3', '2', p3[q3.top()[2]][1] + p2[q3.top()[2]][1]))
    if intersections:
        return min(intersections, key=lambda x: x[3])
    else:
        return None

In [465]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestTriSearchAlgorithms

TestTriSearchAlgorithms("test_valid_paths").test_valid_paths("test_tri_ucs_romania", tridirectional_search)
TestTriSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_tri_ucs_romania", tridirectional_search)
TestTriSearchAlgorithms("test_explored_counts").test_explored_counts("test_tri_ucs_romania", tridirectional_search)

UnitTest passed successfully for "test_tri_ucs_romania.test_valid_paths"!
UnitTest passed successfully for "test_tri_ucs_romania.test_optimal_paths"!
UnitTest passed successfully for "test_tri_ucs_romania.test_explored_counts"!


### Exercise 4: Upgraded Tridirectional search (8 Pts) <a name="tri-upgraded"></a>

This is the heart of the assignment. Implement tridirectional search in such a way as to consistently improve on the
performance of your previous implementation. This means consistently exploring fewer nodes during your search in order
to reduce runtime. Keep in mind, we are not performing 3 bidirectional A* searches. We are searching from each of the goals towards the other two goals, in the direction that seems most promising.

The specifics are up to you, but we have a few suggestions:
 * Tridirectional A*
 * choosing landmarks and pre-computing reach values
 * ATL (A\*, landmarks, and triangle-inequality)
 * shortcuts (skipping nodes with low reach values)

`tridirectional_upgraded()` should return a path between all three nodes.

> **Requirements**
> 1. Implement the `tridirectional_upgraded()` method below. Optionally, you can also implement `compute_landmarks()` and `custom_heuristic()`, but neither of them are absolutely necessary to receive full marks on this exercise.
> 2. If all three goals are the same, simply return an empty list like `[]`.
> 3. If there are 2 identical goals (i.e. `[a,b,b]`) then return the path `[a...b]` (i.e. just the path from `a` to `b`). Do **NOT** call your bidirectional searches for this, your tridirectional search should naturally handle this scenario.
> 4. Return the best path found as a Python list connecting all three goal nodes (inclusive).
> 5. You can obtain a dictionary of the neighbors of a node by calling `graph.neighbors(node)`.
> 6. Sort the neighbors of a node alphabetically before processing them.
> 7. You can access the edge weight between two nodes `u` and `v` by using `graph.get_edge_weight(u, v)`.
> 8. Make sure no node is explored more than once, regardless of which frontier explored it.
> 9. We provide a small margin of error for your explored count when grading.

> **Hints**
> 1. **What makes tridirectional A* different from tridirectional UCS?** Well for starters, A* uses heuristics to accelerate the exploration process. While the Euclidean heuristic is still available, there are new two goals for every search that is expanding, which means there will be two heuristic values. Which one should you choose? Could we combine them? Its time to get creative!
> 2. **What are Landmarks/Reaches/Shortcuts?** Cool stuff! Checkout the textbook and the provided resources, they are definitely worth reading.
> 3. **How does `compute_landmarks` work?** Seeing a lot of concern over this, `compute_landmarks` is optional and is a pre-processing function where we pass you the test graph and you can compute landmark information for it. The return value is a `list` of **four** elements, and each element is suggested to contain an identifier for what that landmark is along with a map/dictionary kind of thing that contains information about each node's true path cost to that landmark. We will first call your `compute_landmarks` function to get your landmarks, then we will reset the graph (so that your `compute_landmarks` function doesn’t blow up your explored count) then we call your tridirectional function and pass in your landmarks.

In [ ]:
#export

def compute_landmarks(graph):
    """
    (Optional)
    Feel free to implement this method for computing landmarks. We will call
    tridirectional_upgraded() with the object returned from this function.

    Args:
        graph (ExplorableGraph): Undirected graph to search.

    Returns:
    List with not more than 4 computed landmarks. 
    """
    # TODO: finish this function͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    return None


def custom_heuristic(graph, u, v):
    """
        Feel free to use this method to try and work with different heuristics and come up with a better search algorithm.
        Args:
            graph (ExplorableGraph): Undirected graph to search.
            u (str): Key for the first node to calculate from.
            v (str): Key for the second node to calculate to.
        Returns:
            Custom heuristic distance between `v` node and `goal` node
        """
    pass


def tridirectional_upgraded(graph, goals, heuristic=euclidean_dist_heuristic, landmarks=None):
    """
    Exercise 4: Upgraded Tridirectional Search

    See README.MD for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        goals (list): Key values for the 3 goals
        heuristic: Function to determine distance heuristic.
            Default: euclidean_dist_heuristic.
        landmarks: Iterable containing landmarks pre-computed in compute_landmarks()
            Default: None

    Returns:
        The best path as a list from one of the goal nodes (including both of
        the other goal nodes).
    """
    # TODO: finish this function͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    raise NotImplementedError

In [ ]:
# Local tests͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
from utilities.localtests import TestTriSearchAlgorithms

landmarks = TestTriSearchAlgorithms("get_landmarks").get_landmarks(compute_landmarks)
TestTriSearchAlgorithms("test_valid_paths").test_valid_paths("test_tri_upgraded_romania", tridirectional_upgraded, heuristic=euclidean_dist_heuristic, landmarks=landmarks)
TestTriSearchAlgorithms("test_optimal_paths").test_optimal_paths("test_tri_upgraded_romania", tridirectional_upgraded, heuristic=euclidean_dist_heuristic, landmarks=landmarks)
TestTriSearchAlgorithms("test_explored_counts").test_explored_counts("test_tri_upgraded_romania", tridirectional_upgraded, heuristic=euclidean_dist_heuristic, landmarks=landmarks)

## Extra Credit - The Race <a name="race"></a>

Here's your chance to show us your best stuff. This is a **two goal search** on the **Atlanta** map. It is a race, so there is a leaderboard on Gradescope that will help determine who gets how many extra credit points.

To participate in the leaderboard, implement `custom_search()` using whatever strategy you like. Recognize that this is optional and for extra credit and is not needed for the regular A1 submission (in fact it is a whole separate Gradescope submission with a separate deadline).

**Specific details will be posted soon on Edstem.**

**Bonus points are added to the grade for this assignment, not to your overall grade.**

We have included the "Haversine" heuristic below. All of the local tests on the Atlanta map use this method. For the race, you can use whatever you choose, but know that the Atlanta map positions are (latitude, longitude). If you would like to learn more about this formula, here is a link: https://en.wikipedia.org/wiki/Haversine_formula

In [ ]:
#export

def haversine_dist_heuristic(graph, v, goal):
    """
    Note: This provided heuristic is for the Atlanta race.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        v (str): Key for the node to calculate from.
        goal (str): Key for the end node to calculate to.

    Returns:
        Haversine distance between `v` node and `goal` node
    """

    #Load latitude and longitude coordinates in radians:͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    vLatLong = (math.radians(graph.nodes[v]["pos"][0]), math.radians(graph.nodes[v]["pos"][1]))
    goalLatLong = (math.radians(graph.nodes[goal]["pos"][0]), math.radians(graph.nodes[goal]["pos"][1]))

    #Now we want to execute portions of the formula:͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    constOutFront = 2*6371 #Radius of Earth is 6,371 kilometers
    term1InSqrt = (math.sin((goalLatLong[0]-vLatLong[0])/2))**2 #First term inside sqrt
    term2InSqrt = math.cos(vLatLong[0])*math.cos(goalLatLong[0])*((math.sin((goalLatLong[1]-vLatLong[1])/2))**2) #Second term
    return constOutFront*math.asin(math.sqrt(term1InSqrt+term2InSqrt)) #Straight application of formula

In [ ]:
#export

def load_data(graph, time_left):
    """
    Feel free to implement this method. We'll call it only once 
    at the beginning of the Race, and we'll pass the output to your custom_search function.
    graph: a networkx graph
    time_left: function you can call to keep track of your remaining time.
        usage: time_left() returns the time left in milliseconds.
        the max time will be 10 minutes.

    * To get a list of nodes, use graph.nodes()
    * To get node neighbors, use graph.neighbors(node)
    * To get edge weight, use graph.get_edge_weight(node1, node2)
    """

    # nodes = graph.nodes()͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    return None

In [ ]:
#export

def custom_search(graph, start, goal, data=None):
    """
    Race!: Implement your best search algorithm here to compete against the
    other student agents.

    If you implement this function and submit your code to Gradescope, you'll be
    registered for the Race!

    See README.md for exercise description.

    Args:
        graph (ExplorableGraph): Undirected graph to search.
        start (str): Key for the start node.
        goal (str): Key for the end node.
        data :  Data used in the custom search.
            Will be passed your data from load_data(graph).
            Default: None.

    Returns:
        The best path as a list from the start to the goal node (including both).
    """

    # TODO: finish this function!͏󠄂͏️͏󠄌͏󠄎͏󠄎͏󠄊͏󠄁
    raise NotImplementedError